In [1]:
!unzip data.zip

Archive:  data.zip
   creating: data/
   creating: data/Development/
  inflating: data/Development/dev_definitions.txt  
  inflating: data/Development/dev_examples.txt  
  inflating: data/Development/dev_hypernyms.txt  
  inflating: data/Development/dev_labels.txt  
  inflating: data/README.txt         
   creating: data/Test/
  inflating: data/Test/test_definitions.txt  
  inflating: data/Test/test_examples.txt  
  inflating: data/Test/test_hypernyms.txt  
   creating: data/Training/
  inflating: data/Training/train_definitions.txt  
  inflating: data/Training/train_examples.txt  
  inflating: data/Training/train_hypernyms.txt  
  inflating: data/Training/train_labels.txt  


In [2]:
!pip install tensorflow_text
!pip install transformers

     |████████████████████████████████| 4.9 MB 4.1 MB/s 
     |████████████████████████████████| 462 kB 48.3 MB/s 
     |████████████████████████████████| 4.0 MB 4.4 MB/s 
     |████████████████████████████████| 895 kB 45.3 MB/s 
     |████████████████████████████████| 77 kB 6.7 MB/s 
     |████████████████████████████████| 596 kB 46.7 MB/s 
     |████████████████████████████████| 6.5 MB 31.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
import re
import os
import errno
import random
import pandas as pd

from sklearn import metrics

import warnings
warnings.filterwarnings('ignore')

from sklearn.utils import shuffle

!rm -rf processed_data

try:
    os.makedirs("processed_data")
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

In [4]:
def decontracted(phrase):

    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can't", "can not", phrase)
    phrase = re.sub(r"gonna", "going to", phrase)
    phrase = re.sub(r"wanna", "want to", phrase)

    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    phrase = re.sub(r"\.+", " ", phrase)
    phrase = re.sub(r"[^A-Za-z$]", " ", phrase)
    phrase = re.sub(r" +", " ", phrase)

    return phrase

In [5]:
def get_df(pref, testing=False):

    dataset = []
    with open(pref + "_" + "examples.txt") as examples:
        for example in examples:
            example = example[:-1].lower().split("\t")
            if example[0] != decontracted(example[0]):
                text = re.sub(example[0], "\$", example[2])
                text = decontracted(text)
                text = re.sub(r"\$", example[0], text)
                example[2] = text
            else:
                example[2] = decontracted(example[2])
            dataset.append(example)

    with open(pref + "_" + "hypernyms.txt") as hypernyms:
        for index, line in enumerate(hypernyms):
            line = line[:-1].lower().split("\t")
            line = " ; ".join(line)
            line = re.sub(r'_', ' ', line)
            dataset[index].append(line)

    with open(pref + "_" + "definitions.txt") as definitions:
        for index, line in enumerate(definitions):
            line = line[:-1].lower().split(";")
            for i in range(len(line)):
                line[i] = line[i].strip()
                if dataset[index][0] != decontracted(dataset[index][0]):
                    text = re.sub(dataset[index][0], "\$", line[i])
                    text = decontracted(text)
                    text = re.sub(r"\$", dataset[index][0], text)
                    line[i] = text
                else:
                    line[i] = decontracted(line[i])
            line = " ; ".join(line)
            dataset[index].append(line)

    cols = ["target", "position", "sentence", "hypernym", "definition"]

    if not testing:
        cols.append("label")
        with open(pref + "_" + "labels.txt") as labels:
            for index, line in enumerate(labels):
                line = line[:-1]
                dataset[index].append(line)

    df = pd.DataFrame(dataset, columns=cols)

    return df

In [6]:
dev_df = get_df("data/Development/dev")
train_df = get_df("data/Training/train")
test_df = get_df("data/Test/test", True)

dev_df.to_csv("processed_data/dev.csv", index=None)
train_df.to_csv("processed_data/train.csv", index=None)
test_df.to_csv("processed_data/test.csv", index=None)

In [7]:
def augment(filename):

    df = pd.read_csv(filename)

    n = df.shape[0]

    for i in range(n):
        row = df.iloc[i]
        if row['label'] == 'T':
            if not isinstance(row['hypernym'], str):
                continue
            position = row['position']
            sentence = row['sentence']
            hypernyms = row['hypernym'].split(' ; ')
            target = row['target']
            definition = row['definition']
            label = row['label']
            for hypernym in hypernyms:
                new_row = {'target': hypernym, 'position': position, 'sentence': sentence.replace(target, hypernym), 
                            'hypernym': target, 'definition': definition, 'label': label}
                df = df.append(new_row, ignore_index = True)
        
        while True:
            random_row = df.iloc[random.randint(0, n - 1)]
            if random_row['target'] != row['target']:
                position = row['position']
                sentence = row['sentence']
                target = row['target']
                hypernyms = random_row['hypernym']
                definition = random_row['definition']
                label = 'F'
                new_row = {'target': target, 'position': position, 'sentence': sentence, 
                                'hypernym': hypernym, 'definition': definition, 'label': label}
                df = df.append(new_row, ignore_index = True)
                break

    df = shuffle(df)
    df.reset_index(drop = True, inplace = True)
    return df

In [8]:
aug_train_df = augment('processed_data/train.csv')
aug_train_df.to_csv('processed_data/train_aug.csv')

In [9]:
import tensorflow_text
import tensorflow as tf
import tensorflow_hub as hub

import numpy as np
import pandas as pd

from keras import backend as K

import transformers
transformers.logging.set_verbosity_error()

from transformers import BertTokenizer
from tensorflow.keras.optimizers import Adam

In [10]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

transformers.logging.set_verbosity_error()

def extract_data(filename, test_data = False):

    df = pd.read_csv(filename)
    X = {'input_type_ids':[],'input_word_ids':[],'input_mask':[], 'mask1': [], 'mask2': []}
    Y = []

    for _, row in df.iterrows():

        word = row['target']
        sentence = row['sentence']
        hypernyms = row['hypernym']
        definitions = row['definition']

        if not test_data:
          if row['label'] == 'T':
              label = 1.0
          else:
              label = 0.0

        desc = [definitions]

        if isinstance(hypernyms, str):
          desc.append(hypernyms)
        
        desc = ' ; '.join(desc)

        sentence = sentence.replace(word.lower(), '$ ' + word + ' $')

        x = tokenizer(sentence, desc, max_length = 128, padding='max_length', truncation = True)

        sep1 = x['input_ids'].index(102)
        sep2 = x['input_ids'].index(102, sep1 + 1)

        x['mask1'] = np.zeros((128))
        x['mask2'] = np.zeros((128))

        dol1 = x['input_ids'].index(1002)
        dol2 = x['input_ids'].index(1002, dol1 + 1)

        for i in range(1, sep1):
            x['mask1'][i] = 1.0
        
        for i in range(sep1 + 1, sep2):
            x['mask2'][i] = 1.0

        for i in range(dol1, dol2 + 1):
            x['token_type_ids'][i] = 1.0
            x['mask2'][i] = 1.0

        X['input_type_ids'].append(x['token_type_ids'])
        X['input_word_ids'].append(x['input_ids'])
        X['input_mask'].append(x['attention_mask'])
        X['mask1'].append(x['mask1'])
        X['mask2'].append(x['mask2'])

        if not test_data:
          Y.append(label)     

    X['input_type_ids'] = np.array(X['input_type_ids'], dtype=np.float32)
    X['input_word_ids'] = np.array(X['input_word_ids'], dtype=np.float32)
    X['input_mask'] = np.array(X['input_mask'], dtype=np.float32)

    X['mask1'] = np.array(X['mask1'], dtype=np.float32)
    X['mask2'] = np.array(X['mask2'], dtype=np.float32)

    Y = np.array(Y)

    return X, Y

X,Y = extract_data('processed_data/train_aug.csv')
Xval, Yval = extract_data('processed_data/dev.csv')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [11]:
# Input layers -----------------------------

input_type_ids = tf.keras.layers.Input(shape = (128), dtype = tf.int32, name = "input_type_ids")
input_word_ids = tf.keras.layers.Input(shape = (128), dtype = tf.int32, name = "input_word_ids")
input_mask = tf.keras.layers.Input(shape = (128), dtype = tf.int32, name = "input_mask")

input_sent_mask = tf.keras.layers.Input(shape=(128), dtype = tf.float32, name = "input_sent_mask")
input_desc_mask = tf.keras.layers.Input(shape=(128), dtype = tf.float32, name = "input_desc_mask")

# -------------------------------------------


# Encoder -----------------------------------

encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2", name = "bert-encoder", trainable = True)
pooled_output, sequence_output = encoder([input_word_ids, input_mask, input_type_ids])

# -------------------------------------------


# Decoder -----------------------------------

input_sent_mask = tf.expand_dims(input_sent_mask, axis = 2)
sent_output = tf.multiply(input_sent_mask, sequence_output)
sent_output = tf.keras.layers.Masking(mask_value = 0)(sent_output)
sent_output = tf.keras.layers.LSTM(32, activation = 'relu')(sent_output)

input_desc_mask = tf.expand_dims(input_desc_mask, axis = 2)
desc_output = tf.multiply(input_desc_mask, sequence_output)
desc_output = tf.keras.layers.Masking(mask_value = 0)(desc_output)
desc_output = tf.keras.layers.LSTM(32, activation = 'relu')(desc_output)

pooled_output = tf.keras.layers.Dense(32, activation = 'relu')(pooled_output)

concat = tf.keras.layers.Concatenate(name = "concatenate")([sent_output, desc_output, pooled_output])
concat = tf.keras.layers.Dropout(0.3)(concat)
output = tf.keras.layers.Dense(1, activation = 'sigmoid', name = "output")(concat)

# -------------------------------------------

# Encoder-Decoder Model ---------------------

model = tf.keras.models.Model(
      inputs = [
        input_word_ids,
        input_mask,
        input_type_ids,
        input_sent_mask,
        input_desc_mask], 
      outputs = output)

model.compile(optimizer=Adam(learning_rate = 5e-6), loss="binary_crossentropy", metrics=["accuracy"])

# -------------------------------------------

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 128)]        0           []                               
                                                                                                  
 input_type_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 input_1 (InputLayer)           [(None, 128, 1)]     0           []                               
                                                                                              

In [12]:
def find_optimal_threshold(target, predicted):
    fpr, tpr, thresholds = metrics.roc_curve(target, predicted)
    fpr = fpr[1:]
    tpr = tpr[1:]
    thresholds = thresholds[1:]
    x = tpr - fpr
    idx = np.argmax(x)
    return thresholds[idx]

class roc_auc_threshold(tf.keras.callbacks.Callback):

    def __init__(self, training_data, validation_data):
        self.x = training_data[0]
        self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]

    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        model.save('model.' + str(epoch) + '.h5')
        y_pred_val = self.model.predict(self.x_val)
        optimal_threshold = find_optimal_threshold(self.y_val, y_pred_val)
        y_pred_val = y_pred_val.flatten()
        y_truth_val = self.y_val.flatten()
        val_correct = 0
        val_total = len(y_pred_val)
        for i in range(len(y_pred_val)):
          pred = 0
          if y_pred_val[i] >= optimal_threshold:
            pred = 1
          if pred == y_truth_val[i]:
            val_correct += 1
        print(' optimal_threshold:', round(optimal_threshold, 5), '-',  
              'val_accuracy_optimal:', round(val_correct / val_total, 5))
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

cb = roc_auc_threshold(training_data = ([X['input_word_ids'], X['input_mask'], X['input_type_ids'], X['mask1'], X['mask2']], Y),
                  validation_data = ([Xval['input_word_ids'], Xval['input_mask'], Xval['input_type_ids'], Xval['mask1'], Xval['mask2']], Yval))

In [13]:
history = model.fit([X['input_word_ids'], X['input_mask'], X['input_type_ids'], X['mask1'], X['mask2']], Y, 
          epochs = 4, batch_size = 16, callbacks=[cb], verbose = 1,
          validation_data = ([Xval['input_word_ids'], Xval['input_mask'], Xval['input_type_ids'], Xval['mask1'], Xval['mask2']], Yval))

Epoch 1/4
433/433 [==============================] - 741s 2s/step - loss: 0.5308 - accuracy: 0.7238 - val_loss: 0.6563 - val_accuracy: 0.6735
Epoch 2/4
433/433 [==============================] - 713s 2s/step - loss: 0.3161 - accuracy: 0.8682 - val_loss: 0.7045 - val_accuracy: 0.7095
Epoch 3/4
433/433 [==============================] - 717s 2s/step - loss: 0.1837 - accuracy: 0.9339 - val_loss: 0.8076 - val_accuracy: 0.7121
Epoch 4/4
433/433 [==============================] - 719s 2s/step - loss: 0.1120 - accuracy: 0.9614 - val_loss: 1.0542 - val_accuracy: 0.7095


In [20]:
epoch = 1
pretrained_model = tf.keras.models.load_model('model.' + str(epoch) + '.h5', custom_objects={'KerasLayer': hub.KerasLayer})

In [21]:
Xtest, Ytest = extract_data('processed_data/test.csv', True)
y_predicted = pretrained_model.predict([Xtest['input_word_ids'], Xtest['input_mask'], Xtest['input_type_ids'], Xtest['mask1'], Xtest['mask2']])
y_predicted = y_predicted.flatten()

In [22]:
optimal_threshold = 0.7198

file = open('all_output.txt', 'w')
for val in y_predicted:
  if val < optimal_threshold:
    print('F',file=file)
  else:
    print('T',file=file)
file.close()